## LLM 개선 방법
### Fine tuning : Data 기반 재학습, People Digitalization
### Knowledge Base : Vector DB를 활용한 정보 증강, Fact(Information) Based Service

- Project : 글을 Input 했을 때, MidJourny Style Prompt로 Return

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.

In [ ]:
from huggingface_hub import login
login('') # input token

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16

)
model_id = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def get_com(query : str, model, tokenizer, max_tokens=1000) -> str:
  device = "cuda:0"


  prompt_tem = '''
  <bos><start_of_turn>user
  {query}
  <end_of_turn>
  <start_of_turn>model
  '''
  #  <start_of_turn>user
  # {query}
  # <end_of_turn>
  # <start_of_turn>\n<start_of_turn>model
  prompt = prompt_tem.format(query=query)

  encoders = tokenizer(prompt, return_tensors="pt",add_special_tokens=True)

  model_inputs = encoders.to(device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=max_tokens,
      do_sample=True)
      # pad_token_id=tokenizer.eos.token_id)

  decode = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decode)

In [ ]:
result = get_com(query = "'Customer 테이블에서 Sales의 카운트가 3이상인 컬럼'만 가져오는 SQL",model=model,tokenizer=tokenizer)
print(result)


  user
  'Customer 테이블에서 Sales의 카운트가 3이상인 컬럼'만 가져오는 SQL
  
  model
    ```sql
  SELECT Column_Name
  FROM Customer
  WHERE Sales > 3;
  ```

**where Clause**

* `Column_Name` is the name of the column you want to select that has the counts.
* `Sales` is the name of the column that contains the counts.
* `>` operator is used to filter the results where the `Sales` column has a value greater than 3.

**Example:**

Suppose the `Customer` table has the following data:

| ID | Name | Sales |
|---|---|---|
| 1 | John | 5 |
| 2 | Mary | 7 |
| 3 | Peter | 8 |
| 4 | Sarah | 10 |
| 5 | David | 3 |

The query would return the following results:

| ID | Name | Sales |
|---|---|---|
| 3 | Peter | 8 |
| 4 | Sarah | 10 |


## 문제점
### 1. 현재, 3이상이지만 >3으로 표현
### 2. 카운트 값을 원했으나 value를 기준으로 쿼리를 작성


## Fine Tuning 방법

### 허깅페이스 Data로 Fine Tuning